In [0]:
!pip install xxhash

     |████████████████████████████████| 193 kB 6.5 MB/s 
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-4e805ea6-e9e3-4abb-9dbe-dc790ea9ce78/bin/python -m pip install --upgrade pip' command.


In [0]:

from pyspark.sql import functions as F
# Step 3: Define your data
data = [(1, 101, "John", "Doe", 25, "Engineer"),
        (2, 102, "Alice", "Smith", 30, "Data Scientist"),
        (3, 103, "Bob", "Johnson", 28, "Analyst")]

# Step 4: Define the schema for your DataFrame
schema = ["id", "clientid", "First_Name", "Last_Name", "Age", "Occupation"]
schema2 = ["id2", "clientid2", "First_Name2", "Last_Name2", "Age2", "Occupation2"]


targetdf = spark.createDataFrame(data,schema)
targetdf.display()



# Step 3: Define your data
data2 = [(1, 101, "satyam", "patel", 23, "Engineer"),
        (4, 104, "Alica", "Smith", 30, "Data Scientist"),
        (5, 105, "jassi", "Johnson", 28, "Analyst")]

sourcedf = spark.createDataFrame(data2,schema2)
sourcedf.show()


# #left join 
# joineddf = sourcedf.alias("a").join(targetdf.alias("b"), sourcedf["ID"] == targetdf["ID"],"left")


# filtereddf = joineddf.filter(xxhash64(joineddf.id,joineddf.clientid,joineddf.First_Name,joineddf.Last_Name,joineddf.Age,joineddf.Occupation) != xxhash64(joineddf.id2,joineddf.clientid2,joineddf.First_Name2,joineddf.Last_Name2,joineddf.Age2,joineddf.Occupation2))

# filteredf.show()



filtereddf = sourcedf.alias("a").join(targetdf.alias("b"), sourcedf["id"] == targetdf["id2"], "left").filter(
    (F.col("a.clientid") != F.col("b.clientid2")) |
    (F.col("a.First_Name") != F.col("b.First_Name2")) |
    (F.col("a.Last_Name") != F.col("b.Last_Name2")) |
    (F.col("a.Age") != F.col("b.Age2")) |
    (F.col("a.Occupation") != F.col("b.Occupation2"))
)

# Step 3: Show the resulting DataFrame
filtereddf.show()


id,clientid,First_Name,Last_Name,Age,Occupation
1,101,John,Doe,25,Engineer
2,102,Alice,Smith,30,Data Scientist
3,103,Bob,Johnson,28,Analyst


+---+---------+-----------+----------+----+--------------+
|id2|clientid2|First_Name2|Last_Name2|Age2|   Occupation2|
+---+---------+-----------+----------+----+--------------+
|  1|      101|     satyam|     patel|  23|      Engineer|
|  4|      104|      Alica|     Smith|  30|Data Scientist|
|  5|      105|      jassi|   Johnson|  28|       Analyst|
+---+---------+-----------+----------+----+--------------+

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/databricks/python/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<command-979170347664277>", line 36, in <module>
    filtereddf = sourcedf.alias("a").join(targetdf.alias("b"), sourcedf["id"] == targetdf["id2"], "left").filter(
  File "/databricks/spark/python/pyspark/instrumentation_utils.py", line 48, in wrapper
    res = func(*args, **kwargs)
  File "/databricks/spark/python/pyspark/sql/dataframe.py", line 2918, in __getitem__
    jc = self._jdf.apply(item)
  File "/databricks/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1321, in __call__
    return_value = get_return_value(
  File "/databricks/spark/python/pyspark/errors/exceptions.py", line 234, in deco
    raise converted from None
pyspark.errors.exceptions.AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `id` cannot be

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col
from pyspark.sql.functions import concat, lit

# Step 1: Create or get a Spark session
spark = SparkSession.builder.appName("JoinAndFilterExample").getOrCreate()

# Step 2: Define your data
data = [(1, 101, "John", "Doe", 25, "Engineer","Y","2022-05-15","9999-12-31"),
        (2, 102, "Alice", "Smith", 30, "Data Scientist","Y","2022-06-15","9999-12-31"),
        (3, 103, "Bob", "Johnson", 28, "Analyst","Y","2022-05-15","9999-12-31")]

# Step 3: Define the schema for your DataFrame
schema = ["id", "clientid", "First_Name", "Last_Name", "Age", "Occupation","active_status","start_date","end_date"]
schema2 = ["id2", "clientid2", "First_Name2", "Last_Name2", "Age2", "Occupation2"]

targetdf = spark.createDataFrame(data, schema)
#targetdf.show()



#targetdf.write.format("delta").mode("append").save("/tmp/delta/target")
targetdf.write.format("delta").mode("overwrite").saveAsTable("Target2")

# Step 4: Define your data
data2 = [(1, 101, "satyam", "patel", 23, "Engineer"),
         (4, 104, "Alica", "Smith", 30, "Data Scientist"),
         (5, 105, "jassi", "Johnson", 28, "Analyst"),
         (1,101,"satyam","verma",22,"engineer")]

sourcedf = spark.createDataFrame(data2, schema2)
#sourcedf.show()
# ... (Previous code)

filtereddf = sourcedf.alias("a").join(targetdf.alias("b"), col("a.id2") == col("b.id"), "left")

filtereddf = filtereddf.select(sourcedf["*"],targetdf.id,targetdf.clientid,targetdf.First_Name,targetdf.Last_Name,targetdf.Age,targetdf.Occupation)


filteredDf = filtereddf.filter(xxhash64(filtereddf.))

filtereddf.display()


mergeDF = filtereddf.withColumn("MERGEKEY",concat(filtereddf.id,filtereddf.clientid))
mergeDF.display()

dummydf = mergeDF.filter(col("MERGEKEY").isNotNull()).withColumn("MERGEKEY", lit(None))
dummydf.display()





scddf = mergeDF.union(dummydf)
scddf.display()








id2,clientid2,First_Name2,Last_Name2,Age2,Occupation2,id,clientid,First_Name,Last_Name,Age,Occupation
1,101,satyam,patel,23,Engineer,1,101,John,Doe,25,Engineer
4,104,Alica,Smith,30,Data Scientist,null,null,null,null,null,null
5,105,jassi,Johnson,28,Analyst,null,null,null,null,null,null
1,101,satyam,verma,22,engineer,1,101,John,Doe,25,Engineer


id2,clientid2,First_Name2,Last_Name2,Age2,Occupation2,id,clientid,First_Name,Last_Name,Age,Occupation,MERGEKEY
1,101,satyam,patel,23,Engineer,1,101,John,Doe,25,Engineer,1101
4,104,Alica,Smith,30,Data Scientist,null,null,null,null,null,null,null
5,105,jassi,Johnson,28,Analyst,null,null,null,null,null,null,null
1,101,satyam,verma,22,engineer,1,101,John,Doe,25,Engineer,1101


id2,clientid2,First_Name2,Last_Name2,Age2,Occupation2,id,clientid,First_Name,Last_Name,Age,Occupation,MERGEKEY
1,101,satyam,patel,23,Engineer,1,101,John,Doe,25,Engineer,null
1,101,satyam,verma,22,engineer,1,101,John,Doe,25,Engineer,null


id2,clientid2,First_Name2,Last_Name2,Age2,Occupation2,id,clientid,First_Name,Last_Name,Age,Occupation,MERGEKEY
1,101,satyam,patel,23,Engineer,1,101,John,Doe,25,Engineer,1101
4,104,Alica,Smith,30,Data Scientist,null,null,null,null,null,null,null
5,105,jassi,Johnson,28,Analyst,null,null,null,null,null,null,null
1,101,satyam,verma,22,engineer,1,101,John,Doe,25,Engineer,1101
1,101,satyam,patel,23,Engineer,1,101,John,Doe,25,Engineer,null
1,101,satyam,verma,22,engineer,1,101,John,Doe,25,Engineer,null


In [0]:

from delta.tables import DeltaTable
from pyspark.sql.functions import expr

#deltaTable = DeltaTable.forPath(spark, "/tmp/delta/target")
deltaTable = DeltaTable.forName(spark,"Target2")
dfUpdates = scddf
deltaTable.alias('target') \
  .merge(
       dfUpdates.alias('source'),
       condition= "concat(target.id,target.clientid) = source.MERGEKEY and target.active_status = 'Y'"
  ) \
  .whenMatchedUpdate(set =
    {
      "active_status": "'N'",
      "end_date": "current_date"
    }
  ) \
  .whenNotMatchedInsert(values =
    {
   "id":"source.id2",
   "clientid":"source.clientid2",
   "First_Name":"source.First_Name2",
   "Last_Name":"source.Last_Name2",
   "Age":"source.Age2",
   "Occupation":"source.Occupation2",
   "active_status":"'Y'",
   "start_date":"current_date",
   "end_date":"9999-12-31"
    }
  ) \
  .execute()



In [0]:
#spark.sql("select * from Target2")

delta_table_name = "Target2"

# Read the Delta table into a DataFrame
delta_df = spark.read.format("delta").table(delta_table_name)

# Show the data of the Delta table
delta_df.show()

+---+--------+----------+---------+---+--------------+-------------+----------+----------+
| id|clientid|First_Name|Last_Name|Age|    Occupation|active_status|start_date|  end_date|
+---+--------+----------+---------+---+--------------+-------------+----------+----------+
|  2|     102|     Alice|    Smith| 30|Data Scientist|            Y|2022-06-15|9999-12-31|
|  3|     103|       Bob|  Johnson| 28|       Analyst|            Y|2022-05-15|9999-12-31|
|  1|     101|      John|      Doe| 25|      Engineer|            Y|2022-05-15|9999-12-31|
+---+--------+----------+---------+---+--------------+-------------+----------+----------+

